# AirBNB Europa



In [17]:
from pathlib import Path

import csv
import time
import re

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns


%matplotlib inline

## Configuración

Fuente de datos para Europa https://www.kaggle.com/datasets/dipeshkhemani/airbnb-cleaned-europe-dataset


In [18]:
data_folder = Path().cwd().parent / 'data'

europe_file = data_folder / 'airbnb_europe' / 'Aemf1.csv'

## Carga de datos

### Datos Europa

In [19]:
df = pd.read_csv(europe_file)

df.shape

## EDA

### EDA datos Europa

In [20]:
df.info()

In [21]:
df.describe()

In [22]:
df['City'].value_counts()

In [23]:
df['Day'].value_counts()

In [24]:
df['Room Type'].value_counts()

### Limpieza de datos

In [9]:
city_mapping = {
    'Rome': 0,
    'Paris': 1,
    'Lisbon': 2,
    'Athens': 3,
    'Budapest': 4,
    'Vienna': 5,
    'Barcelona': 6,
    'Berlin': 7,
    'Amsterdam': 8
}
df['City'] = df['City'].map(city_mapping)

In [10]:
day_mapping = {
    "Weekday": 0,
    "Weekend": 1,
}

df['Day'] = df['Day'].map(day_mapping)

In [11]:
room_type_mapping = {
    "Entire": 0,
    "Private": 1,
    "Shared": 2,
}

df['Room Type'] = df['Room Type'].map(day_mapping)

In [16]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

plt.matshow(df.corr())
plt.show()
